https://tproger.ru/translations/markov-chains/

# Цепи Маркова для автоматической генерации тектов

## Структура данных Dictogram

Dictogram (dict — встроенный тип данных словарь в Python) будет отображать зависимость между звеньями и их частотой появления в тексте, то есть их распределение. Но при этом она будет обладать нужным нам свойством словаря — время выполнения программы не будет зависеть от объема входных данных, а это значит, мы создаем эффективный алгоритм.

In [1]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
#         list_of_keys = 
        # вывести 'случайный индекс:', random_int
        for key in self.keys():
            index += self[key]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return key

В конструктор структуре Dictogram можно передать любой объект, по которому можно итерироваться. Элементами этого объекта будут слова для инициализации Dictogram, например, все слова из какой-нибудь книги. В данном случае мы ведем подсчет элементов, чтобы для обращения к какому-либо из них не нужно было пробегать каждый раз по всему набору данных.

Мы также сделали две функции для возврата случайного слова. Одна функция выбирает случайный ключ в словаре, а другая, принимая во внимание число появлений каждого слова в тексте, возвращает нужное нам слово.

## Структура цепи Маркова

In [2]:
def make_markov_model(data):
    markov_model = dict()

    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
    return markov_model

В реализации выше у нас есть словарь, который хранит окна в качестве ключа в паре «(ключ, значение)» и распределения в качестве значений в этой паре.

## Cтруктура цепи Маркова N-го порядка

In [3]:
def make_higher_order_markov_model(order, data):
    markov_model = dict()

    for i in range(0, len(data)-order):
        # Создаем окно
        window = tuple(data[i: i+order])
        # Добавляем в словарь
        if window in markov_model:
            # Присоединяем к уже существующему распределению
            markov_model[window].update([data[i+order]])
        else:
            markov_model[window] = Dictogram([data[i+order]])
    return markov_model

## Парсинг модели

Отлично, мы реализовали словарь. Но как теперь совершить генерацию контента, основываясь на текущем состоянии и шаге к следующему состоянию? Пройдемся по нашей модели:

In [4]:
from collections import deque
import re


def generate_random_start(model):
    # Чтобы сгенерировать любое начальное слово, раскомментируйте строку:
    # return random.choice(model.keys())

    # Чтобы сгенерировать "правильное" начальное слово, используйте код ниже:
    # Правильные начальные слова - это те, что являлись началом предложений в корпусе
    if 'END' in model:
        seed_word = 'END'
        while seed_word == 'END':
            seed_word = model['END'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model):
    current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
    sentence[0] = sentence[0].capitalize()
    return ' '.join(sentence) + '.'
    return sentence

Лучше генерировать «правильные» начальные слова – те, которые и в исходном тексте стояли в начале предложения. Для этого мы в словаре находим все ключи «END» и выбираем слово, следующее за одним из них. После генерации начального слова мы ищем, какое слово может идти дальше, обращаясь к тому же словарю, и выбираем нужное на основании комбинации вероятности и случайности. Продолжаем это делать, пока предложение не достигнет установленной нами длины, и в конце возвращаем его. 

## Примеры

In [5]:
text = """Today you are you. That is truer than true. There is no one
alive who is you-er than you»
«You have brains in your head. You have feet in your shoes.
You can steer yourself any direction you choose. You’re on
your own»
«The more that you read, the more things you will know. The
more that you learn, the more places you’ll go»
«Think left and think right and think low and think high. Oh,
the thinks you can think up if only you try»"""

# text = open('oldman.txt', 'r').read()
text = text.replace("»", " END").replace("«", "").replace("\n", " ").replace(".", " END").replace(",", "")
frags = [f.lower() if f != 'END' else f for f in text.split(" ") if f]
print(frags)

['today', 'you', 'are', 'you', 'END', 'that', 'is', 'truer', 'than', 'true', 'END', 'there', 'is', 'no', 'one', 'alive', 'who', 'is', 'you-er', 'than', 'you', 'END', 'you', 'have', 'brains', 'in', 'your', 'head', 'END', 'you', 'have', 'feet', 'in', 'your', 'shoes', 'END', 'you', 'can', 'steer', 'yourself', 'any', 'direction', 'you', 'choose', 'END', 'you’re', 'on', 'your', 'own', 'END', 'the', 'more', 'that', 'you', 'read', 'the', 'more', 'things', 'you', 'will', 'know', 'END', 'the', 'more', 'that', 'you', 'learn', 'the', 'more', 'places', 'you’ll', 'go', 'END', 'think', 'left', 'and', 'think', 'right', 'and', 'think', 'low', 'and', 'think', 'high', 'END', 'oh', 'the', 'thinks', 'you', 'can', 'think', 'up', 'if', 'only', 'you', 'try', 'END']


### Для 1-го порядка

In [6]:
model = make_markov_model(frags)
model

{'END': {'oh': 1,
  'that': 1,
  'the': 2,
  'there': 1,
  'think': 1,
  'you': 3,
  'you’re': 1},
 'alive': {'who': 1},
 'and': {'think': 3},
 'any': {'direction': 1},
 'are': {'you': 1},
 'brains': {'in': 1},
 'can': {'steer': 1, 'think': 1},
 'choose': {'END': 1},
 'direction': {'you': 1},
 'feet': {'in': 1},
 'go': {'END': 1},
 'have': {'brains': 1, 'feet': 1},
 'head': {'END': 1},
 'high': {'END': 1},
 'if': {'only': 1},
 'in': {'your': 2},
 'is': {'no': 1, 'truer': 1, 'you-er': 1},
 'know': {'END': 1},
 'learn': {'the': 1},
 'left': {'and': 1},
 'low': {'and': 1},
 'more': {'places': 1, 'that': 2, 'things': 1},
 'no': {'one': 1},
 'oh': {'the': 1},
 'on': {'your': 1},
 'one': {'alive': 1},
 'only': {'you': 1},
 'own': {'END': 1},
 'places': {'you’ll': 1},
 'read': {'the': 1},
 'right': {'and': 1},
 'shoes': {'END': 1},
 'steer': {'yourself': 1},
 'than': {'true': 1, 'you': 1},
 'that': {'is': 1, 'you': 2},
 'the': {'more': 4, 'thinks': 1},
 'there': {'is': 1},
 'things': {'you': 

In [7]:
generate_random_sentence(100, model).replace(" END", ".")

'There is you-er than true. you’re on your shoes. you learn the thinks you can think right and think right and think low and think high. you learn the more that is truer than you. you’re on your head. think low and think left and think high. you. there is no one alive who is truer than true. think right and think up if only you choose. you have brains in your shoes. there is you-er than true. there is you-er than true. that you try. you can.'

### Для N-го порядка

In [8]:
model = make_higher_order_markov_model(2, frags)
model

{('END', 'oh'): {'the': 1},
 ('END', 'that'): {'is': 1},
 ('END', 'the'): {'more': 2},
 ('END', 'there'): {'is': 1},
 ('END', 'think'): {'left': 1},
 ('END', 'you'): {'can': 1, 'have': 2},
 ('END', 'you’re'): {'on': 1},
 ('alive', 'who'): {'is': 1},
 ('and', 'think'): {'high': 1, 'low': 1, 'right': 1},
 ('any', 'direction'): {'you': 1},
 ('are', 'you'): {'END': 1},
 ('brains', 'in'): {'your': 1},
 ('can', 'steer'): {'yourself': 1},
 ('can', 'think'): {'up': 1},
 ('choose', 'END'): {'you’re': 1},
 ('direction', 'you'): {'choose': 1},
 ('feet', 'in'): {'your': 1},
 ('go', 'END'): {'think': 1},
 ('have', 'brains'): {'in': 1},
 ('have', 'feet'): {'in': 1},
 ('head', 'END'): {'you': 1},
 ('high', 'END'): {'oh': 1},
 ('if', 'only'): {'you': 1},
 ('in', 'your'): {'head': 1, 'shoes': 1},
 ('is', 'no'): {'one': 1},
 ('is', 'truer'): {'than': 1},
 ('is', 'you-er'): {'than': 1},
 ('know', 'END'): {'the': 1},
 ('learn', 'the'): {'more': 1},
 ('left', 'and'): {'think': 1},
 ('low', 'and'): {'think'